In [10]:
from metpy.interpolate import interpolate_to_grid
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import glob 
from pyproj import Proj

In [7]:
sites = ["atmos", "big", "csu", "down", "hum", "neiu_ccics", "neiu", "nu", "shedd", "uic", "vlpk"]
date = "20250324"
wxt_ds = {}
for s in sites:
    try:
        list = glob.glob(f"{s}-wxt-a1/*{date}*.nc")[0]
        wxt_ds[s] = xr.open_dataset(list)
    except IndexError:
        continue

In [19]:
time_to_interpolate = "2025-03-24T12:00:00"
# Define your projection (example: Lambert Conformal Conic)
proj = Proj(proj='lcc', lat_1=41.9742, lat_2=45, lat_0=41.9742, lon_0=41.9742)

# Example x, y coordinates (in meters)
x_range = np.arange(-50000., 50000., 500.)
y_range = np.arange(-50000., 50000., 500.)
x, y = np.meshgrid(x_range, y_range)

# Convert to lon, lat
lon, lat = proj(x, y, inverse=True)
u = {}
v = {}
for s in sites:
    if s not in wxt_ds.keys():
        continue
    u_wind = wxt_ds[s]["wind_mean_10s"] * -np.cos(np.deg2rad(wxt_ds[s].wind_dir_10s))
    v_wind = wxt_ds[s]["wind_mean_10s"] * -np.sin(np.deg2rad(wxt_ds[s].wind_dir_10s))
    u[s] = u_wind.sel(time=time_to_interpolate, method='nearest').values
    v[s] = v_wind.sel(time=time_to_interpolate, method='nearest').values

In [ ]:
lat = 41.9742
lon = -87.9073
print(f"Latitude: {lat}, Longitude: {lon}")
